In [4]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Flatten, Dropout, Embedding, TimeDistributed
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.utils import np_utils
import pandas as pd
%load_ext tensorboard

#import text file 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
file = open("/Users/mollykammann/Documents/create_collab/firstOutput.txt", encoding= "UTF-8")
lyrics = file.read()
lyrics = lyrics.lower()
print(lyrics)

is it getting better
or do you feel the same?
will it make it easier on you, now
you got someone to blame?

you say one love, one life
when it's one need in the night
one love, we get to share it
leaves you, baby, if you don't care for it

did i disappoint you
or leave a bad taste in your mouth?
you act like you never had love
and you want me to go without

well, it's too late tonight
to drag the past out into the light
we're one, but we're not the same
we get to carry each other, carry each other
one

have you come here for forgiveness?
have you come to raise the dead?
have you come here to play jesus
to the lepers in your head?

did i ask too much? more than a lot
you gave me nothing, now it's all i got
we're one, but we're not the same
well, we hurt each other, then we do it again

you say love is a temple, love a higher law
love is a temple, love the higher law
you ask me to enter, but then you make me crawl
and i can't be holding on to what you got
when all you got is hurt

one lo

In [6]:
chars = sorted(list(set(lyrics)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# finding num. of characters in text & number of unique characters
n_characters = len(lyrics)
n_vocabulary = len(chars)

print("Total characters in lyric set: " , n_characters)
print("Total vocabulary in lyric set: " , n_vocabulary)

Total characters in lyric set:  115811
Total vocabulary in lyric set:  84


In [7]:
seqlen = 100
data_x = []
data_y = []

for i in range(0, n_characters - seqlen, 1):
    seq_in = lyrics[i:i+seqlen]
    seq_out = lyrics[i + seqlen]
    data_x.append([char_indices[char] for char in seq_in])
    data_y.append(char_indices[seq_out])
n_patterns = len(data_x)
print( 'Total Patterns : ', n_patterns)

Total Patterns :  115711


In [8]:
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_x , (n_patterns, seqlen, 1))
# Normalizing input data :
X = X/ float(n_vocabulary)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

In [9]:
LSTM_layer_num = 4
layer_size = [256,256,256,256] 
model = Sequential()
model.add(LSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))
for i in range(1,LSTM_layer_num) :
    model.add(LSTM(layer_size[i], return_sequences=True))
model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 84)                2150484   
_________________________________________________________________
activation (Activation)      (None, 84)               

In [10]:
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

In [41]:
# Fit the model :
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])

Epoch 1/30
724/724 [==============================] - ETA: 0s - loss: 3.0377
Epoch 00001: loss improved from inf to 3.03765, saving model to Weights-LSTM-improvement-001-3.03765-bigger.hdf5
724/724 [==============================] - 1109s 2s/step - loss: 3.0377 - val_loss: 3.0433
Epoch 2/30
724/724 [==============================] - ETA: 0s - loss: 3.0056
Epoch 00002: loss improved from 3.03765 to 3.00558, saving model to Weights-LSTM-improvement-002-3.00558-bigger.hdf5
724/724 [==============================] - 1119s 2s/step - loss: 3.0056 - val_loss: 3.0116
Epoch 3/30
724/724 [==============================] - ETA: 0s - loss: 2.8184
Epoch 00003: loss improved from 3.00558 to 2.81839, saving model to Weights-LSTM-improvement-003-2.81839-bigger.hdf5
724/724 [==============================] - 1253s 2s/step - loss: 2.8184 - val_loss: 2.8338
Epoch 4/30
724/724 [==============================] - ETA: 0s - loss: 2.6854
Epoch 00004: loss improved from 2.81839 to 2.68539, saving model to Weig

In [15]:
# Load wights file :
weights_file = './Weights-LSTM-improvement-028-0.06100-bigger.hdf5' # weights file path
model.load_weights(weights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
# set a random seed :
start = np.random.randint(0, len(data_x)-1)
pattern = data_x[start]
print('Seed : ')
print("\"",''.join([indices_char[value] for value in pattern]), "\"\n")

# How many characters you want to generate
generated_characters = 300

# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocabulary)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = indices_char[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')

Seed : 
"  outside
(you're gonna meet her there)
a man makes a car
(she's your destination)
and builds a road  "

atiiet in a sooe tee prores rhe oights io whthonh i want to likn the the buhr feac the raadit br you, ther ir
ivss the saalow oun lodh
ht aeeire io the miogea 
you say on oo the one
ori foedu hn the saado

in the worl of rie cace in the nettine in your bireitidt to may
fo l sati ii the wasssb

llvea
Done


In [5]:


#step = seqlen
#sentences = []
#for i in range(0, len(text) - seqlen - 1, step):
   # sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences): 
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
   
    
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
59/60 [============================>.] - ETA: 0s - loss: 2.1744 - categorical_crossentropy: 2.1744 - accuracy: 0.5427
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "                     I don't know, Honey"
                     I don't know, Honey                                                                                                                                                                                                                                                                                                                                                                                                                
----- diversity: 0.5
----- Generating with seed: "                     I don't know, Honey"
                     I don't know, Honey                                                                                                                                                                         

                                                                BUTCH
                         I never in a starts on it can in a some the 
                                            VINCENT
                                      
----- diversity: 1.0
----- Generating with seed: "the box the Cap'n 
               Crunc"
the box the Cap'n 
               Crunch –... in the two show face and 
               Wo we're buydo look lioggen is.

               SLESHER in a sittobo?

                                     A don't get in a cartave of that 
               VINCENT
                                          back unconnels your far as dour on a life?

               car a marto! Jive Butch a gix and so smy 
               WILLICA1E JOMY YOLR
60/60 [==============================] - 39s 654ms/step - loss: 0.8815 - categorical_crossentropy: 0.8815 - accuracy: 0.7581
Epoch 10/50
59/60 [============================>.] - ETA: 0s - loss: 0.8566 - categorical_crossentropy: 0.8566 - accuracy: 

                                                                                                                                                                                                                                                                                                                                                                                           
----- diversity: 0.5
----- Generating with seed: "N
                         About fiftee"
N
                         About fifteen man as is a big shit.

                                                                                                                                                                                           VINCENT
                         I don't know what that was a fuck your 
                         I don't know the little of yours. We 
                         was a given and – who 
----- diversity: 1.0
----- Generating with seed: "N
                         About fiftee"
N
 

Epoch 27/50
59/60 [============================>.] - ETA: 0s - loss: 0.6867 - categorical_crossentropy: 0.6866 - accuracy: 0.8061
----- Generating text after Epoch: 26
----- diversity: 0.2
----- Generating with seed: " You always say that, the same thing 
 "
 You always say that, the same thing 
                         the should be fuckin' dead the back of 
                         the should be suilf you think we he see 
                         the barrel of herone, do the two men looks like this 
                         this shit on a too but and Jules, who's 
                         the bar, of your granddad one of 
                         the bare face a second the two men l
----- diversity: 0.5
----- Generating with seed: " You always say that, the same thing 
 "
 You always say that, the same thing 
                         want the fuck your fighte. You want the 
                         gonna give a should place you 
                         to the two men like they cross

                         We had the sinuts with a severses. The 
                         and the sime the car. Here looked to the 
               and the same fuckin' with me, I don't 
                         down the same corfules, who with the 
                         this her her get this compil. An Ain TRISS the register 
                         the shit.

               Butch looks at him.

   
----- diversity: 1.0
----- Generating with seed: "               THE WOLF
               "
               THE WOLF
                         Wherobagen after freak, like that.

                                 Butch Dodding thrount wan this is fight!" 
                         if what who want the see theands who disto your try your she're they just 
                         go wim the car, MARSELLUS topcanders. But 
                         forment.

               They should be fuckin' never vasime.

             
60/60 [==============================] - 40s 666ms/step - loss: 0.6720 

                                                                                                                                                                                                                                                                                                                                                                                                                  
----- diversity: 0.5
----- Generating with seed: " 
                         midst.

  "
 
                         midst.

                                                                                                                                                                                                                                                                                                                                                                                                                  
----- diversity: 1.0
----- Generating with seed: " 
                  


                           
60/60 [==============================] - 41s 685ms/step - loss: 0.6346 - categorical_crossentropy: 0.6346 - accuracy: 0.8195
Epoch 49/50
59/60 [============================>.] - ETA: 0s - loss: 0.6327 - categorical_crossentropy: 0.6327 - accuracy: 0.8197
----- Generating text after Epoch: 48
----- diversity: 0.2
----- Generating with seed: "                   FABIENNE (O.S.)
    "
                   FABIENNE (O.S.)
                         You want to go out of the bathroom door 
                         and I want you to know how not so 
                         the bathroom door.

                                     VINCENT
                         What we're to controtherfucker.

                                     VINCENT
                         What do you turk it was a firded in 
              
----- diversity: 0.5
----- Generating with seed: "                   FABIENNE (O.S.)
    "
                   FABIENNE (O.S.)
                              

In [12]:

    
for diversity in [0.5]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[1486:1526]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.


        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)



----- diversity: 0.5
----- Generating with seed: "YOUNG MAN
                         No, "
YOUNG MAN
                         No, that I'm don't think I said it 
                         this shit into TV prothing the register.

                                                                               JULES
                         I'm thing. The works to his first this this 
                         motherfucker.

                                                                    FABIENNE
                         What's here, do the you this go in you, Deaning fat ther arba? He the but fast. 
                                              BUTCH
                         Mia morns out the should be a senter.

                                                                                                                                                                                                                                                                                     

In [13]:
for diversity in [0.5]:
    print('----- diversity:', diversity)

    generated = 'THsbv aksdlfa alksd oqosibd fkl: si jkpq'

    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(5000):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----- diversity: 0.5
----- Generating with seed: "       (crocliffrested shitumagemering, "
THsbv aksdlfa alksd oqosibd fkl: si jkpqbut faster 
                                           FABIENNE
                         You will be a shit out those phone shotga 
                         be or the through them, the garbage band his finters this 
                                     VINCENT
                         What do you thisle of the car. And it's 
                         this nerd a faction.

                                                                                                                                                                                                                                                                                                                                        VINCENT
                              (on the chas the boys)
                         You wanna did to should that this morning.

                                    